In [21]:
import json
import warnings
import pandas as pd
from sec_api import QueryApi, XbrlApi

In [ ]:
api_key = "e3d46df2271d81a4b7a71153aef74fa5e7dae956314122b155767267425b36a6"

# Query 20 most recent 10Q Info

In [ ]:
# queryAPI = QueryApi(api_key=api_key)

In [ ]:
# query = {"query": {
#             "query_string": {
#                 "query": "ticker:TSLA AND formType:\"10-Q\"",
#                     }
#                 },
#          "from": "0",
#          "size": 4*5,
#          "sort": [{"filedAt": {"order": "desc"}}]
#             }

In [ ]:
# response = queryAPI.get_filings(query)
# response_parsed = pd.DataFrame.from_records(response["filings"])

In [ ]:
# response_parsed.to_csv("TSLA_10Q_INFO_2017Q1_2023Q2.csv", index=False)
response_parsed = pd.read_csv("TSLA_10Q_INFO_2017Q1_2023Q2.csv")

# Parse most recent 10Q

In [ ]:
# htm_url = response_parsed.loc[0,"linkToFilingDetails"]

In [ ]:
# xbrlAPI = XbrlApi(api_key)

In [ ]:
# xbrl_json = xbrlAPI.xbrl_to_json(htm_url)

In [ ]:
# with open("tesla_10Q_2023Q2.json", "w") as f:
#     json.dump(xbrl_json, f)

In [2]:
with open("tesla_10Q_2023Q2.json", "r") as f:
    xbrl_json = json.load(f)

In [3]:
xbrl_json["CoverPage"]["DocumentPeriodEndDate"]

'2023-06-30'

In [4]:
BalanceSheets = xbrl_json["BalanceSheets"]
StatementsOfIncome = xbrl_json["StatementsOfIncome"]
StatementsOfCashFlows = xbrl_json["StatementsOfCashFlows"]
StatementsOfShareholdersEquity = xbrl_json["StatementsOfShareholdersEquity"]

## Balance Sheets

In [5]:
BalanceSheets.keys()

dict_keys(['CashAndCashEquivalentsAtCarryingValue', 'ShortTermInvestments', 'AccountsReceivableNetCurrent', 'InventoryNet', 'PrepaidExpenseAndOtherAssetsCurrent', 'AssetsCurrent', 'DeferredCostsLeasingNetNoncurrent', 'LeasedAssetsNet', 'PropertyPlantAndEquipmentNet', 'OperatingLeaseRightOfUseAsset', 'DigitalAssetsNetNonCurrent', 'IntangibleAssetsNetExcludingGoodwill', 'Goodwill', 'OtherAssetsNoncurrent', 'Assets', 'AccountsPayableCurrent', 'AccruedAndOtherCurrentLiabilities', 'ContractWithCustomerLiabilityCurrent', 'CustomerDepositsLiabilitiesCurrent', 'LongTermDebtAndFinanceLeasesCurrent', 'LiabilitiesCurrent', 'LongTermDebtAndFinanceLeasesNoncurrent', 'ContractWithCustomerLiabilityNoncurrent', 'OtherLiabilitiesNoncurrent', 'Liabilities', 'CommitmentsAndContingencies', 'RedeemableNoncontrollingInterestEquityCarryingAmount', 'PreferredStockValue', 'CommonStockValue', 'AdditionalPaidInCapitalCommonStock', 'AccumulatedOtherComprehensiveIncomeLossNetOfTax', 'RetainedEarningsAccumulatedDef

In [32]:
def balancesheets_to_df(BalanceSheets):
    """
    """
    warnings.filterwarnings('ignore', message=".*A value is trying to be set on a copy.*")
    df_list = []

    for name in BalanceSheets.keys():
        try:
            df = pd.DataFrame.from_records(BalanceSheets[name])
            if "segment" in df.columns:
                df = df.query("segment.isna()")
            df["period"] = df["period"].apply(lambda x: x["instant"])
            df["name"] = name
            df = df[["name", "period", "value"]]
            df_list.append(df)
        except Exception as ex:
            print(name, "---", ex)
    
    return pd.concat(df_list, axis=0)

In [33]:
balancesheets_to_df(BalanceSheets)

CommitmentsAndContingencies --- "['value'] not in index"


C:\Users\mushj\AppData\Local\Temp\ipykernel_36320\2078047896.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["period"] = df["period"].apply(lambda x: x["instant"])
C:\Users\mushj\AppData\Local\Temp\ipykernel_36320\2078047896.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["name"] = name
C:\Users\mushj\AppData\Local\Temp\ipykernel_36320\2078047896.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

,name,period,value
0,CashAndCashEquivalentsAtCarryingValue,2023-06-30,15296000000
1,CashAndCashEquivalentsAtCarryingValue,2022-12-31,16253000000
2,CashAndCashEquivalentsAtCarryingValue,2022-06-30,18324000000
3,CashAndCashEquivalentsAtCarryingValue,2021-12-31,17576000000
0,ShortTermInvestments,2023-06-30,7779000000
...,...,...,...
1,StockholdersEquity,2022-12-31,44704000000
0,MinorityInterest,2023-06-30,764000000
1,MinorityInterest,2022-12-31,785000000
0,LiabilitiesAndStockholdersEquity,2023-06-30,90591000000
